# Cavity Quality and Losses by the Energy Participation Ratio (EPR) method
Calculation of resonant cavity quality factors, losses, life-time and more using the EPR method. This code is based on [Ansys HFSS](https://www.ansys.com/products/electronics/ansys-hfss) and the [pyEPR library](https://github.com/zlatko-minev/pyEPR).

The cavity is half of a [Tesla cavity](https://arxiv.org/pdf/physics/0003011.pdf), this will be usefule in a later notebook where we compare the full Tesla cavity to our (half Tesla) cavity.

#### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True

In [2]:
import sys
import numpy as np
from IPython.display import display, Math, Latex, display_markdown
from pathlib import Path
import pandas as pd
from scipy import constants

import pyEPR as epr
from pyEPR.calcs import Convert
from pyEPR.core import *
from pyEPR.ansys import *
import warnings
warnings.simplefilter("ignore")

In [3]:
path_to_project = 'D:\\Users\\Daniel\\pyEPR-Test\\SRF_cavity'

### 🔷 Mode analysis

#### 🔹 Connect to HFSS

In [4]:
pinfo = epr.Project_Info(project_path = path_to_project, 
                         project_name = 'model',
                         design_name  = 'dirt loss')

INFO 07:33PM [connect]: Connecting to Ansys Desktop API...
INFO 07:33PM [load_ansys_project]: 	File path to HFSS project found.
INFO 07:33PM [load_ansys_project]: 	Opened Ansys App
INFO 07:33PM [load_ansys_project]: 	Opened Ansys Desktop v2020.1.0
INFO 07:33PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/Daniel/
	Project:   model
INFO 07:33PM [connect]: 	Opened active design
	Design:    dirt loss [Solution type: Eigenmode]
INFO 07:33PM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 07:33PM [connect]: 	Connection to Ansys established successfully. 😀 



#### 🔹 Define non-linear objects

In [5]:
pinfo.junctions['j1'] = {'Lj_variable' : 'Lj_1', 
                         'rect'        : 'rect_jj1', 
                         'line'        : 'line_jj1', 
                         'length'      : epr.parse_units('10um')}

pinfo.validate_junction_info()  
pinfo.dissipative.dielectrics_bulk = ['chip'] 

In [6]:
pinfo.setup.analyze()
eprh = epr.DistributedAnalysis(pinfo)

INFO 07:33PM [analyze]: Analyzing setup Setup1


Design "dirt loss" info:
	# eigenmodes    2
	# variations    1


#### 🔹 Get HFSS mode and quality results

In [7]:
modes      = eprh.get_freqs_bare_pd(eprh.variations[0])
Fs, Qs     = np.array(modes['Freq. (GHz)']), np.array(modes['Quality Factor'])  # Get freqs and Q-factors
mode_names = ['cavity','transmon']
n_modes    = int(pinfo.setup.n_modes)
display(modes)

,Freq. (GHz),Quality Factor
mode,,
0,4.849791,1.150770e+11
1,6.714454,3.895691e+06


#### 🔹 EPR analysis

In [8]:
eprh.do_EPR_analysis(variations=eprh.variations[0]);


Variation 0  [1/1]

  Mode 0 at 4.85 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.0%  2.507e-19 2.507e-19

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.54%
	j1              8.02087e-06  (+)        1.19165e-07
		(U_tot_cap-U_tot_ind)/mean=0.00%
Calculating Qdielectric_chip for mode 0 (0/1)
p_dielectric_chip_0 = 2.16557054700636e-05

  Mode 1 at 6.71 GHz   [2/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               96.8%  1.153e-24 3.695e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.23%
	j1              0.965846  (+)        0.0275048
		(U_tot_cap-U_tot_ind)/mean=1.46%
Calculating Qdielect

#### 🔹 Quantum analysis

In [9]:
epra = epr.QuantumAnalysis(eprh.data_filename)
quantum_results = epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 15, variations=eprh.variations[0], print_result=False);

WARNING 07:46PM [__init__]: <p>Error: <class 'IndexError'></p>


	 Differences in variations:


0, Starting the diagonalization
Finished the diagonalization


#### 🔹 Calculate the EPRs of the modes

In [10]:
eprh.set_mode(0) # cavity mode
p_dielectic_cavity, (ℰ_substr, ℰ_total) = eprh.calc_p_electric_volume('chip')

eprh.set_mode(1) # transmon mode
p_dielectic_transmon, (ℰ_substr, ℰ_total) = eprh.calc_p_electric_volume('chip')

p_dielectrics = np.array([p_dielectic_cavity, p_dielectic_transmon])

print(f' 🔸 Cavity energy in sapphire substrate    = {100*p_dielectic_cavity:.3f}%')
print(f' 🔸 Transmon energy in sapphire substrate  = {100*p_dielectic_transmon:.3f}%')

 🔸 Cavity energy in sapphire substrate    = 0.002%
 🔸 Transmon energy in sapphire substrate  = 64.174%


### 🔷 Life-times

**Life-time from HFSS**

In [11]:
Fs_Hz  = np.array(Convert.toSI(Fs,'GHz'))  # Mode freqs in Hz
omegas = 2*np.pi*Fs_Hz  # Freqs to angular freqs
taus   = Qs/omegas  # Life times

for n in range(n_modes):  # Loop over all modes
    print(f' 🔸 Life-time of {mode_names[n]} mode = {taus[n]*1e3:.3f} ms')

 🔸 Life-time of cavity mode = 3776.467 ms
 🔸 Life-time of transmon mode = 0.092 ms


**Life-time from EPR**

In [12]:
tan_sapp = 4e-7  # Loss tangent of sapphire

tau_epr = lambda p, tan, omega: 1/(p*tan*omega)  # Easily calculate life time with EPR

tau_cavity, tau_transmon = tau_epr(p_dielectrics, tan_sapp, omegas)

print(f' 🔸 Cavity life-time   = {tau_cavity*1e6:.2f} ns')
print(f' 🔸 Transmon life-time = {tau_transmon*1e6:.2f} ns')

 🔸 Cavity life-time   = 3788477.85 ns
 🔸 Transmon life-time = 92.34 ns


### 🔷 Losses

#### 🔹 Chip surface loss

In [13]:
t        = 3e-9
eps      = 10
tan_surf = 5e-3
for n in range(n_modes):
    print(f'  {mode_names[n]}\n'+'═'*(len(mode_names[n])+4))
    eprh.set_mode(n)

    # --- Surface integral ---
    E_surf = 0
    for surf in ['chip_bottom','chip_top']:
        calcobject = CalcObject([], eprh.setup)
        vecE = calcobject.getQty("E").smooth()
        A = vecE.times_eps()
        B = vecE.conj()
        A = A.dot(B)
        A = A.real()
        A = A.integrate_surf(name=surf)

        E_subs = A.evaluate(lv=eprh._get_lv()) 
        E_surf += E_subs*t*eps

    # --- Volume integral ---
    E_total = eprh.calc_energy_electric(smooth=True)

    p_surf = E_surf/E_total      # EPR of surface 
    Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
    tau_surf = Q_surf/omegas[n]  #  Life-time of surface
    

    print(f' 🔸 EPR on chip surface    = {100*p_surf:.8f}%')
    print(f' 🔸 Q-factor chip surface  = {1e-6*Q_surf:.1f} M')
    print(f' 🔸 Life-time chip surface = {1e3*tau_surf:.4f} ms\n')


  cavity
══════════
 🔸 EPR on chip surface    = 0.00000072%
 🔸 Q-factor chip surface  = 27764.4 M
 🔸 Life-time chip surface = 911.1414 ms

  transmon
════════════
 🔸 EPR on chip surface    = 0.02050689%
 🔸 Q-factor chip surface  = 1.0 M
 🔸 Life-time chip surface = 0.0231 ms



#### 🔹 Cavity surface loss

In [15]:
for n in range(n_modes):
    print(f'  {mode_names[n]}\n'+'═'*(len(mode_names[n])+4))
    eprh.set_mode(n)

    # --- Surface integral ---
    surf = 'cavity_1'
    calcobject = CalcObject([], eprh.setup)
    vecE = calcobject.getQty("E").smooth()
    A = vecE.times_eps()
    B = vecE.conj()
    A = A.dot(B)
    A = A.real()
    A = A.integrate_surf(name=surf)

    E_subs = A.evaluate(lv=eprh._get_lv()) 
    E_surf = E_subs*t*eps

    # --- Volume integral ---
    E_total = eprh.calc_energy_electric(smooth=True)

    p_surf = E_surf/E_total      # EPR of surface 
    Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
    tau_surf = Q_surf/omegas[n]  #  Life-time of surface
    

    print(f' 🔸 EPR on cav surface    = {100*p_surf:.8f}%')
    print(f' 🔸 Q-factor cav surface  = {1e-6*Q_surf:.1f} M')
    print(f' 🔸 Life-time cav surface = {1e3*tau_surf:.4f} ms\n')


  cavity
══════════
 🔸 EPR on cav surface    = 0.00040158%
 🔸 Q-factor cav surface  = 49.8 M
 🔸 Life-time cav surface = 1.6344 ms

  transmon
════════════
 🔸 EPR on cav surface    = 0.00820800%
 🔸 Q-factor cav surface  = 2.4 M
 🔸 Life-time cav surface = 0.0578 ms



#### 🔹 Dirt (volume) loss

In [17]:
# Dirt is simulated as much thicker than it actually is (for computation reason). 
# Beacuase of that we reduce the loss tangent to an 'effective loss tangent' which is loss_tan*thick_factor
thick_factor = 1
tan_dirt     = 0.1  # dirt loss tangent

for n in range(n_modes):  # Loop over all the modes
    print(f'  {mode_names[n]}\n'+'═'*(len(mode_names[n])+4))
    eprh.set_mode(n)
    p_dirt, (ℰ_dirt, ℰ_total) = eprh.calc_p_electric_volume('dirt_vol')
    Q_dirt = 1/(thick_factor*tan_dirt*p_dirt)
    tau_dirt = Q_dirt/omegas[n]
    
    print(f'  🔸 EPR of dirt    = {p_dirt:.2e} ( {ℰ_dirt:.2e} / {ℰ_total:.2e} )')
    print(f'  🔸 Quality factor = {Q_dirt:.2e}')
    print(f'  🔸 life time      = {tau_dirt:.2e} μs\n')

  cavity
══════════
  🔸 EPR of dirt    = 6.88e-08 ( 3.45e-26 / 5.01e-19 )
  🔸 Quality factor = 1.45e+08
  🔸 life time      = 4.77e-03 μs

  transmon
════════════
  🔸 EPR of dirt    = 4.58e-12 ( 1.06e-35 / 2.31e-24 )
  🔸 Quality factor = 2.19e+12
  🔸 life time      = 5.18e+01 μs



In [ ]:
# pinfo.disconnect()